# Setting

## 라이브러리 로딩 및 드라이브 연결

In [1]:
from os.path import join

modelPath='/content/drive/MyDrive/'
dataPath='/content/drive/MyDrive/'

dPath=join(dataPath,"example.tsv")





In [2]:
from google.colab import drive
isMount=!df |grep /content/drive
if len(isMount) < 1 : drive.mount('/content/drive', force_remount=False)


In [3]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-3.0.2-py3-none-any.whl (769 kB)
  Using cached tokenizers-0.8.1rc1-cp38-cp38-manylinux1_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Successfully uninstalled tokenizers-0.10.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.8.1
    Uninstalling transformers-4.8.1:
      Successfully uninstalled transformers-4.8.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pickle
from os.path import join,isfile,isdir


In [5]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model


#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
#GPU 사용
device = torch.device("cuda:0")

## 함수 정의

In [6]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [7]:
def get_1d(l):
  ll=[]
  for i in l:
    for j in i:
      ll.append(j)
  return ll

get_1d([[1],[2]])  

[1, 2]

In [8]:
def calc_accuracy2(x,y):
  xx=get_1d(x)
  yy=get_1d(y)
  xl=len(xx)
  cnt=0
  if xl==len(yy):
    for i in range(xl):
      if xx[i] == yy[i]:
        cnt+=1
    return cnt/xl

calc_accuracy2([[1,2],[3,4]],[[1],[2,3,1]])


0.75

In [9]:
def isEqual(X,Y):
  max_vals, max_indices = torch.max(X, 1)
  if max_indices.cpu().numpy()[0] == Y:
    return 1
  else:
    return 0

# Data and Model 준비


## Data Loading

In [10]:
from os.path import join,isfile,isdir


import pandas as pd
if not isfile(dPath) :
  !pwd
  url = 'https://raw.githubusercontent.com/Finfra/AI_NLP/main/7.DeepLearningForNLP_Advanced/example.tsv'
  storage_options = {'User-Agent': 'Mozilla/5.0'}
  df = pd.read_table(url)
  df.to_csv(dPath,index=False,sep="\t")
else:
  df = pd.read_csv(dPath,sep="\t")
df

,id,document,label
0,96,인생 영화 추천,1
1,99,망작이다,0
2,78,쓰래기 보는데 돈을 쓰다니,0
3,95,진짜 재미 없다,0
4,15,볼일 없다,0
5,5,마음이 편해진다.,1
6,43,돈이 아깝다,0
7,71,지루하다,0
8,78,차라리 남중구 강의를 듣겠다,0


In [11]:



data_list = []
for q, label in zip(df['document'], df['label'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

data_list    

[['인생 영화 추천', '1'],
 ['망작이다', '0'],
 ['쓰래기 보는데 돈을 쓰다니', '0'],
 ['진짜 재미 없다', '0'],
 ['볼일 없다', '0'],
 ['마음이 편해진다.', '1'],
 ['돈이 아깝다', '0'],
 ['지루하다', '0'],
 ['차라리 남중구 강의를 듣겠다', '0']]

## Data 전처리

In [12]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
# dataset_train, dataset_test = train_test_split(data_list, test_size=0, random_state=0)
dataset_train=data_list
dataset_test=data_list.copy()

num_classes =df.label.unique().shape[0]
print(num_classes)

2


## Model Loading

In [13]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [14]:
isBertFail=False
isVocFail=False

modelPathBert=join(modelPath,'bertmodel')

if isfile(modelPathBert):
  bertmodel, vocab = get_pytorch_kobert_model()
  model = BERTClassifier(bertmodel,  dr_rate=0.5, num_classes=num_classes ).to(device)
  model.load_state_dict(torch.load(modelPathBert )) 
  print('Model loaded.....')
else:  
  print('No Model loaded.....')
  isBertFail=True


modelPathVocab=join(modelPath,'vocab')
if isfile(modelPathVocab):
  with open(modelPathVocab, "rb") as file_to_read:
    vocab = pickle.load(file_to_read)
  print('Model loaded.....')
else:    
  print('No Model loaded.....')
  isVocFail=True



if isBertFail or isVocFail:
  bertmodel, vocab = get_pytorch_kobert_model()
  
 
model = BERTClassifier(bertmodel,  dr_rate=0.5, num_classes=num_classes ).to(device)

if isBertFail:
  torch.save(model.state_dict(),modelPathBert )

if isVocFail:
  with open(modelPathVocab, "wb") as file_to_save:
    pickle.dump(vocab,file_to_save)


using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
Model loaded.....
Model loaded.....


# Model Training

In [15]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [16]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 50
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5


In [17]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [18]:
data_train[0]

(array([   2, 3774, 3394, 4559,    3,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(5, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 1)

In [19]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [20]:
#BERT 모델 불러오기
# model = BERTClassifier(bertmodel,num_classes=2,  dr_rate=0.5).to(device)
model = BERTClassifier(bertmodel,num_classes=num_classes,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [21]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-21-480b6a139979>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/1 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.6942881345748901 train acc 0.4444444444444444
epoch 1 train acc 0.4444444444444444


<ipython-input-21-480b6a139979>:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/1 [00:00<?, ?it/s]

...

epoch 49 test acc 1.0


  0%|          | 0/1 [00:00<?, ?it/s]

epoch 50 batch id 1 loss 0.034480735659599304 train acc 1.0
epoch 50 train acc 1.0


  0%|          | 0/1 [00:00<?, ?it/s]

epoch 50 test acc 1.0


## Model Save

In [22]:
# Model Save
torch.save(model.state_dict(),modelPathBert )

In [23]:
# Model Load
model = BERTClassifier(bertmodel,  dr_rate=0.5, num_classes=num_classes ).to(device)
model.load_state_dict(torch.load(modelPathBert )) 



<All keys matched successfully>

# Model Implementation 

In [24]:
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

i=0
# cnt=0
test_acc=0
labels=[]
results=[]
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
      i+=1
      token_ids = token_ids.long().to(device)
      segment_ids = segment_ids.long().to(device)
      valid_length= valid_length
      label = label.long().to(device)
      out = model(token_ids, valid_length, segment_ids)
      acc=calc_accuracy(out, label)
      results.append(torch.max(out, 1)[1].cpu().numpy().tolist())
      labels.append(label.cpu().numpy().tolist())
      # print(calc_accuracy(out, label))

      # test_acc += calc_accuracy(out, label)
# lable1=label.cpu().numpy()
result=torch.max(out, 1)[1].cpu().numpy()
print('-----------------')

print("test_acc %: ",calc_accuracy2(labels,results))
print("label  : ", labels)
print('result : ', results )



<ipython-input-24-024e7792074f>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/1 [00:00<?, ?it/s]

-----------------
test_acc %:  1.0
label  :  [[1, 0, 0, 0, 0, 1, 0, 0, 0]]
result :  [[1, 0, 0, 0, 0, 1, 0, 0, 0]]


In [25]:

#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)


using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [26]:
def predict(predict_sentences):
  # predict_sentence="의사"
  dataset_another = [[predict_sentence, '0'] for predict_sentence in predict_sentences]
  # dataset_another = [data]

  another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
  test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

  model.eval()

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)

    valid_length= valid_length
    label = label.long().to(device)

    out = model(token_ids, valid_length, segment_ids)


    test_eval=[]
    for i in out:
      logits=i
      logits = logits.detach().cpu().numpy()
      test_eval.append(np.argmax(logits))
  return test_eval



In [27]:
predict(['볼일 없다','거지 같다','마음이 편해지는 추천작이다. ','인생 추천'])

[0, 0, 1, 1]

In [28]:
# https://tutorials.pytorch.kr/beginner/saving_loading_models.html